# Claim Processing

* We create a tool to help adjusters process claims with text and image
* We will use tecton features to statically and dynamically enrich the context for the LLM
* We will retrieve information from a vector db
* We will show how to leverage ML models in GenAI workflows

**The example claim is a real case, it happened last year, my car was totaled**

## Setup

In [ ]:
from tecton_gen_ai.testing import set_dev_mode, make_local_batch_feature_view, print_md
from tecton_gen_ai.testing.interactive import qna
from tecton_gen_ai.api import Agent, prompt, tool

set_dev_mode()
llm = {"model":"openai/gpt-4o", "temperature":0}

## Build a naive AI agent

In [ ]:
agent = Agent(
    "agent",
    prompt = "You will evaluate a car accident based on the report from someone involved, you need to generate a detailed report for an adjuster to make further decisions",
    llm = llm,
)

print_md(agent.invoke("""
I have an accident at 10am, Feb 20, 2024 near Seattle Chinatown.
I am driving my subaru, and the other driver drove on the bus lane and hit me when I tried to turn left into the garage.
"""))

## Knowing the customer

### Define a feature view to serve policies

| policy_id* | policy |
| -- | -- |

In [ ]:
policy="""
Policy ID: p6909379
Policyholder Information:
- Name: John Doe
- Address: 123 Main Street, San Francisco, CA, USA
- Phone Number: (555) 123-4567
- Email: john.doe@example.com
Vehicle Information:
1. Vehicle 1:
   - Make: Toyota
   - Model: Camry
   - Year: 2020
   - VIN: 1HGBH41JXMN109186
2. Vehicle 2:
   - Make: Subaru
   - Model: Impreza
   - Year: 2014
   - VIN: 2FAGP9CW5EH123456
Coverage Details:
1. Liability Coverage:
   - Bodily Injury Liability: $100,000 per person / $300,000 per accident
   - Property Damage Liability: $50,000 per accident
2. Personal Injury Protection (PIP):
   - Medical Expenses: $10,000 per person
3. Uninsured/Underinsured Motorist Coverage:
   - Bodily Injury: $100,000 per person / $300,000 per accident
4. Comprehensive Coverage:
   - Deductible: $500
   - Coverage Limit: Actual cash value of the vehicle
5. Collision Coverage:
   - Deductible: $500
   - Coverage Limit: Actual cash value of the vehicle
6. Additional Coverages:
   - Rental Reimbursement: $30 per day, up to 30 days
   - Roadside Assistance: Included
Policy Period:
- Effective Date: January 1, 2024
- Expiration Date: December 31, 2024
Premium Information:
- Total Annual Premium: $1,800
- Payment Plan: Monthly ($150 per month)
"""

policy_view = make_local_batch_feature_view(
    "policy_view",
    [{"policy_id": "p6909379", "policy": policy}],
    entity_keys = ["policy_id"],
    description="policy details"
)

### Define policy enriched prompt

In [ ]:
@prompt(sources=[policy_view])
def sys_prompt(policy_view:dict) -> str:
    return f"""You will evaluate a car accident based on the report from the policy holder who involved, you need to generate a detailed report for an adjuster to make further decisions

    Here is the policy detail:

    {policy_view['policy']}
    """

### Use the prompt in the agent

In [ ]:
agent = Agent(
    "agent",
    prompt = sys_prompt,
    llm = "openai/gpt-4o",
)

qna(agent, query="""
I have an accident at 10am, Feb 20, 2024 near Seattle Chinatown.
I am driving my subaru, and the other driver drove on the bus lane and hit me when I tried to turn left into the garage.
""", context={"policy_id":"p6909379"}, diagram=True)

## Structured output

In [ ]:
from pydantic import BaseModel, Field

class Accident(BaseModel):
    date_time: str = Field(description="Date and time in isoformat of the accident, set to empty if not mentioned")
    city: str = Field(description="City where the accident occurred, set to empty if not mentioned")
    vehicle_vin: str = Field(description="VIN number of the policy holder's car involved in this accident, VIN number is in the policy details")
    other_vehicles: list[str] = Field(description="Other vehicles involved in the accident, each should be in the format make-model-plate")

    collision_description: str = Field(description="Description of how the collision occurred")
    in_policy_vehicle_damage_description: str = Field(description="Description of the damage to the insured vehicle")
    other_vehicles_damage_description: str = Field(description="Description of the damage to other vehicles")
    
    next_steps: list[str] = Field(description="Instructions of next steps and things to consider for the adjuster")

agent = Agent(
    "report_eval",
    prompt = sys_prompt,
    llm = llm,
    output_schema = Accident,
)

qna(agent, query="""
I have an accident at 10am, Feb 20, 2024 near Seattle Chinatown.
I am driving my subaru, and the other driver drove on the bus lane and hit me when I tried to turn left into the garage.
""", context={"policy_id":"p6909379"}, diagram=True)

## Understanding images and other multimedia sources

Here are the pictures I took after the accident:

In [ ]:
from IPython.display import display
from PIL import Image

display(Image.open("/Users/han/Downloads/IMG_0483.jpg").resize((512,512)))
display(Image.open("/Users/han/Downloads/IMG_0484.jpg").resize((512,512)))
display(Image.open("/Users/han/Downloads/IMG_0488.jpg").resize((512,512)))


### How to realize multi-modal?

Using special syntax + tool calling can easily achieve multi-modal.

For example if we define the syntax `[img:<id>]` to refer to an image, and then define a python function (as the tool) to retrieve the image data from the id and use a LLM to convert it to text inside the same function. Then the top level LLM is able to use the tool to understand images.

### Define the image tool

In [ ]:
def encode_img(img_id):
    from PIL import Image
    import base64
    from io import BytesIO

    new_size=(512, 512)
    image_path = f"/Users/han/Downloads/IMG_{img_id}.jpg"
    with Image.open(image_path) as img:
        # Resize the image
        resized_img = img.resize(new_size)
        # Create a BytesIO buffer to hold the image data
        buffered = BytesIO()
        # Save the resized image to the buffer in the desired format (e.g., JPEG)
        resized_img.save(buffered, format="JPEG")
        # Get the byte data from the buffer
        img_bytes = buffered.getvalue()
        # Encode the byte data to base64
        img_base64 = base64.b64encode(img_bytes).decode('utf-8')
        return img_base64
        
def describe_image(image_id:str) -> str:
    """
    Describe a image that is related with an car accident.
    
    Args:

        image_id: the id of the image in the format: `[img:<image_id>]`, for example, `[img:abc]` the id is `abc`

    Return:

        The full description of the image that can be helpful to understand the accident
    """
    from openai import OpenAI

    prompt = """Describe a image that is related with an car accident.
    It should have the details of the damage, it should include the car brand, color, and plate and vin if available.
    """
    
    client = OpenAI()
    base64_image = encode_img(image_id)

    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": prompt},
                    {
                        "type": "image_url",
                        "image_url": {
                            "url": f"data:image/jpeg;base64,{base64_image}"
                        }
                    },
                ],
            }
        ],
        max_tokens=1000,
        temperature=0.0,
    )

    return response.choices[0].message.content

### Let the agent use the image tool (when necessary)

In [ ]:
agent = Agent(
    "agent",
    prompt = sys_prompt,
    tools = [describe_image],
    llm = llm,
    output_schema = Accident,
)

qna(agent, query="""
I have an accident at 10am, Feb 20, 2024 near Seattle Chinatown.
I am driving my subaru, and the other driver drove on the bus lane and hit me when I tried to turn left into the garage.
Here are the pictures of the accident.
My damage: [img:0483]
The dodge: [img:0484] [img:0488]
""", context={"policy_id":"p6909379"}, diagram=True)

## Quantifying damage and loss

LLM is not reliable on quantitative analysis. But we can let the agent use quantitative tools!

### Tecton Features as tools

Market value of a vehicle can be served as a feature view:

| make* | model* | year* | value |
| -- | -- | -- | -- |

In [ ]:
market_value = make_local_batch_feature_view(
    "market_value",
    [
        {"make":"subaru", "model":"impreza", "year":2014, "value":14500.0},
    ],
    entity_keys=["make", "model", "year"],
    description="""
    Get the current market value of certain make model and year, 0 means unknown

    make: str: lower case maker of the car, for example: toyota
    model: str: lower case model of the car, for example: tundra
    year: int: the year of the car, for example: 2020
    """
)

### ML Models as a tools

Repair cost estimation can be done by a trained ML model.

Assuming the model takes
* features directly from the report and the policy (make, model, year, severity)
* features derived from the report and policy (price factor that is state dependent)

We can define a tool `estimate_cost` wrapping the ML model, and it also depends on the feature view `price_factor`

In [ ]:
from tecton import resource_provider
from tecton_gen_ai.utils.runtime import get_runtime_resource

@resource_provider()
def my_model():
    # initialize your model instance/client here
    model = "dummy"
    return model

price_factor = make_local_batch_feature_view(
    "price_factor",
    [
        {"state":"wa", "factor":1.5},
        {"state":"ca", "factor":1.1},
    ],
    entity_keys=["state"],
    description="Price factor for damage estimates"
)

@tool(sources=[price_factor], resource_providers={"my_model":my_model})
def estimate_cost(
    state:str,
    make:str, model:str, year:int, front_damage:int, side_damage:int, back_damage:int,
    price_factor,
) -> float:
    """
    Estimate the repair cost of the damage of a car

    Args:

        state: state abbr in lower case, for example: ca, wa, il. It should be the state where the accident happened
        make: make of the car
        model: model of the car
        year: year of the car
        front_damage: front damage severity, 0: no damage, 1: minor, 2: severe
        side_damage: side damage severity, 0: no damage, 1: minor, 2: severe
        back_damage: back damage severity, 0: no damage, 1: minor, 2: severe

    Return:

        The estimated repair cost
    """
    model = get_runtime_resource("my_model")
    assert model == "dummy"
    factor = price_factor.get("factor", 1.0)
    if max(front_damage, side_damage, back_damage)==2:
        return 20000*factor
    return 500*factor

### Tell me the numbers!

In [ ]:
class Damage(BaseModel):
    front_damage: int = Field(description="policyholder's vehicle front damage severity, 0: no damage, 1: minor, 2: severe")
    side_damage: int = Field(description="policyholder's vehicle side damage severity, 0: no damage, 1: minor, 2: severe")
    back_damage: int = Field(description="policyholder's vehicle back damage severity, 0: no damage, 1: minor, 2: severe")
    repair_cost: float = Field(description="The repair cost esimate to the policyholder's vehicle")
    market_value: float = Field(description="The current market value of the policyholder's vehicle")
    
agent_damage = Agent(
    "agent_damage",
    prompt = sys_prompt,
    tools = [describe_image, estimate_cost, market_value],
    llm = llm,
    output_schema = Damage,
)

qna(agent_damage, query="""
I have an accident at 10am, Feb 20, 2024 near Seattle Chinatown.
I am driving my subaru, and the other driver drove on the bus lane and hit me when I tried to turn left into the garage.
Here are the pictures of the accident.
My damage: [img:0483]
The dodge: [img:0484] [img:0488]
""", context={"policy_id":"p6909379"}, diagram=True)

## Interpreting violations

Different states have different laws and rules, written in natural language. So RAG can be used as a tool to look up rules and interpret violations.

### Connect to a vector db

In [ ]:
from tecton_gen_ai.api import VectorDB

vdb = VectorDB(
    "lancedb",
    embedding="openai/text-embedding-3-small",
    uri="/tmp/lancedb",
    table_name="wa_rules",
)

vdb.search("left turn rules", filter={"state":"wa"})

### Vector DB as a retrieval tool

In [ ]:
class RuleFilter(BaseModel):
    state: str = Field(description="lower case abbr of state that the rules belongs to, for example: ca, wa")

@vdb.retriever(filter=RuleFilter)
def get_rules_of_the_road(query, filter, result):
    """Get the related rules of the road of a certain state

    The query should contain the description of the accident (under 200 words)
    """
    return "\n\n".join(x["rule_text"] for x in result)

### Show me the violations

In [ ]:
class Violations(BaseModel):
    rules_violated: str = Field("Based on the rules of the road of the state, explain the potential violations of either party inside this accident")

agent = Agent(
    "agent",
    prompt = sys_prompt,
    tools = [describe_image, get_rules_of_the_road],
    llm = llm,
    output_schema = Violations,
)

qna(agent, query="""
I have an accident at 10am, Feb 20, 2024 near Seattle Chinatown.
I am driving my subaru, and the other driver drove on the bus lane and hit me when I tried to turn left into the garage.
Here are the pictures of the accident.
My damage: [img:0483]
The dodge: [img:0484] [img:0488]
""", context={"policy_id":"p6909379"}, diagram=True)

## A full accident report

In [ ]:
class FullAccidentReport(Accident, Damage, Violations):
    pass

agent = Agent(
    "agent",
    prompt = sys_prompt,
    tools = [describe_image, estimate_cost, market_value, get_rules_of_the_road],
    llm = llm,
    output_schema = FullAccidentReport,
)

qna(agent, query="""
I have an accident at 10am, Feb 20, 2024 near Seattle Chinatown.
I am driving my subaru, and the other driver drove on the bus lane and hit me when I tried to turn left into the garage.
Here are the pictures of the accident.
My damage: [img:0483]
The dodge: [img:0484] [img:0488]
""", context={"policy_id":"p6909379"}, diagram=True)

# One last thing

### Tecton agentic system is fully testable, deployable, composable and decomposable